## Loading the Data

In [3]:
from datasets import load_dataset
dataset = load_dataset('billingsmoore/tibetan-to-english-translation-dataset')

/home/j/Desktop/MLotsawa/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
dataset = dataset['train'].train_test_split(.15)

In [5]:
dataset['train'][0]

{'tibetan': 'མི་བཟད་ཆུ་སྲིན་ལྟ་བུའི་སྡེ་ཚོགས་རྣམས།།',
 'phonetic': 'mi zé chusin tabü dé tsok nam',
 'english': 'Out of the turbulent sea of sixteenfold danger'}

## Load Unfinetuned Tokenizer, Model, and Data Collator

In [6]:
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM

checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, device_map="cuda:0")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

## Add Tibetan to Tokenizer

The T5 tokenizer does not notably support the Tibetan script. So, we need to add it manually. Once the characters have been added to the tokenizer, the model needs to have its token embeddings resized to accomodate the added tokens. This is all pretty straightforward, as seen in the code below.

In [7]:
# Tibetan characters to add
tibetan_chars = [
    # Consonants
    "ཀ", "ཁ", "ག", "ང", "ཅ", "ཆ", "ཇ", "ཉ", "ཏ", "ཐ", "ད", "ན", "པ", "པ", "ཕ", "བ", "མ",
    "ཙ", "ཚ", "ཛ", "ཝ", "ཞ", "ཟ", "འ", "ཡ", "ར", "ལ", "ཤ", "ཥ", "ས", "ཧ", "ཨ",

    # Subjoined Consonants
    "ྐ", "ྑ", "ྒ", "ྒྷ", "ྔ", "ྕ", "ྖ", "ྗ", "྘", "ྙ", "ྚ", "ྛ", "ྜ", "ྜྷ", "ྞ", "ྟ",
    "ྠ", "ྡ", "ྡྷ", "ྣ", "ྤ", "ྥ", "ྦ", "ྦྷ", "ྨ", "ྩ", "ྪ", "ྫ", "ྫྷ", "ྭ", "ྮ", "ྯ",
    "ྰ", "ྱ", "ྲ", "ླ", "ྴ", "ྵ", "ྶ", "ྷ", "ྸ", "ྐྵ", "ྺ", "ྻ", "ྼ", "྽", "྾", "྿",

    # Vowels
    "ི", "ཱི", "ུ", "ཱུ", "ྲྀ", "ཷ", "ླྀ", "ཹ", "ེ", "ཻ", "ོ", "ཽ", "ཾ", "ཿ",

    # Other Marks and Symbols
    "འ", "ཡ", "ར", "ལ", "ཤ", "ཥ", "ས", "ཧ", "ཨ",

    # Additional Tibetan Characters
    "ཀྵ", "ཁྵ", "གྵ", "ངྵ", "ཅྵ", "ཆྵ", "ཇྵ", "ཉྵ", "ཏྵ", "ཐྵ", "དྵ", "ནྵ", "པྵ", 
    "པྵ", "ཕྵ", "བྵ", "མྵ", "ཙྵ", "ཚྵ", "ཛྵ", "ཝྵ", "ཞྵ", "ཟྵ", "འྵ", "ཡྵ", "རྵ", 
    "ལྵ", "ཤྵ", "ཥྵ", "སྵ", "ཧྵ", "ཨྵ", "པྪ", "པྫ", "པྫྷ", "པྭ", "པྮ", "པྯ", "པྰ", 
    "པྱ", "པྲ", "པླ", "པྴ", "པྵ", "པྶ", "པྷ", "པྸ", "པྐྵ", "པྺ", "པྻ", "པྼ", "པ྽", 
    "པ྾", "པ྿"
]


#'ཀཁགངཅཆཇཉཏཐདནཔཕབམཙཚཛཝཞཟའཡརལཤཥསཧཨ'

# Add the Tibetan characters to the tokenizer's vocabulary
new_tokens = [char for char in tibetan_chars if char not in tokenizer.get_vocab()]

# Add new tokens to the tokenizer
tokenizer.add_tokens(new_tokens)

# Resize model embeddings to accommodate the new vocabulary size
model.resize_token_embeddings(len(tokenizer))

Embedding(32245, 512)

## Preprocess Data

The dataset can now be tokenized for training.

In [8]:
def translation_preprocess_function(examples):

    # Prepare translation inputs and targets
    translation_inputs = ['Translate Tibetan to English: ' + example for example in examples['tibetan']]
    translation_targets = [example for example in examples['english']]
    
    # Tokenize translation inputs and targets
    translation_model_inputs = tokenizer(translation_inputs, text_target=translation_targets, 
                                         max_length=300, truncation=True, padding="max_length")
    
    
    return translation_model_inputs


In [9]:
def transliteration_preprocess_function(examples):
    # Prepare transliteration inputs and targets
    transliteration_inputs = ['Transliterate: ' + example for example in examples['tibetan']]
    transliteration_targets = [example for example in examples['phonetic']]
    
    # Tokenize transliteration inputs and targets
    transliteration_model_inputs = tokenizer(transliteration_inputs, text_target=transliteration_targets, 
                                             max_length=300, truncation=True, padding="max_length")
    
    return transliteration_model_inputs

In [10]:
translation_tokenized_dataset = dataset.map(translation_preprocess_function, batched=True)

Map: 100%|██████████| 12184/12184 [00:03<00:00, 3372.88 examples/s]


In [11]:
transliteration_tokenized_dataset = dataset.map(transliteration_preprocess_function, batched=True)

Map:   0%|          | 0/69042 [00:00<?, ? examples/s]

Map: 100%|██████████| 12184/12184 [00:03<00:00, 3189.78 examples/s]


In [12]:
from datasets import concatenate_datasets

tokenized_dataset = {}

tokenized_dataset['train'] = concatenate_datasets([translation_tokenized_dataset['train'], transliteration_tokenized_dataset['train']])
tokenized_dataset['test'] = concatenate_datasets([translation_tokenized_dataset['test'], transliteration_tokenized_dataset['test']])

## Train the Model

Finally, we can train the model. Note that the optimizer used is Adafactor. This is the optimizer that is preferred for translation tasks and for the T5 model in general. The transformers api includes a built in version of Adafactor, but I define it separately here so that we can optimize it with the 'accelerate' library.

In [13]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, Adafactor
from accelerate import Accelerator

accelerator = Accelerator()

optimizer = Adafactor(
    model.parameters(), 
    scale_parameter=True, 
    relative_step=False, 
    warmup_init=False, 
    lr=3e-4
)

model, optimizer = accelerator.prepare(model, optimizer)

In [16]:
import numpy as np
import evaluate

metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [12]:
import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: billingsmoore. Use `wandb login --relogin` to force relogin


True

In [17]:
training_args = Seq2SeqTrainingArguments(
    output_dir=f"dual-task-poc",
    auto_find_batch_size=True,
    predict_with_generate=True,
    fp16=False, #check this
    push_to_hub=False,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    num_train_epochs=3
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer,
    optimizers=(optimizer, None),
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: billingsmoore. Use `wandb login --relogin` to force relogin


  1%|          | 501/51783 [01:37<2:40:20,  5.33it/s]

{'loss': 0.4351, 'grad_norm': 0.1823342889547348, 'learning_rate': 0.0002971032964486414, 'epoch': 0.03}


  2%|▏         | 1001/51783 [03:11<2:44:55,  5.13it/s]

{'loss': 0.22, 'grad_norm': 0.3234434127807617, 'learning_rate': 0.00029420659289728285, 'epoch': 0.06}


...

100%|██████████| 51783/51783 [3:09:25<00:00,  5.80it/s]/home/j/Desktop/MLotsawa/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
                                                       
100%|██████████| 51783/51783 [3:23:44<00:00,  5.80it/s]

{'eval_loss': 0.06824082881212234, 'eval_bleu': 26.1597, 'eval_gen_len': 17.0385, 'eval_runtime': 857.5151, 'eval_samples_per_second': 28.417, 'eval_steps_per_second': 3.552, 'epoch': 3.0}


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].
100%|██████████| 51783/51783 [3:23:47<00:00,  4.24it/s]

{'train_runtime': 12228.3308, 'train_samples_per_second': 33.876, 'train_steps_per_second': 4.235, 'train_loss': 0.09931060706740599, 'epoch': 3.0}


TrainOutput(global_step=51783, training_loss=0.09931060706740599, metrics={'train_runtime': 12228.3308, 'train_samples_per_second': 33.876, 'train_steps_per_second': 4.235, 'total_flos': 3.28509444980736e+16, 'train_loss': 0.09931060706740599, 'epoch': 3.0})